In [13]:
%matplotlib inline

import os  
os.chdir('../')

In [14]:
ls data/cifar-10-batches-py/

batches.meta  data_batch_2  data_batch_4  readme.html
data_batch_1  data_batch_3  data_batch_5  test_batch


In [15]:
unpickle_and_encode('data/cifar-10-batches-py/batches.meta')

NameError: name 'raw_image_data' is not defined

In [16]:
import numpy as np
import pickle
from functools import reduce
from matplotlib import pyplot as plt

In [17]:
def load_files(input_dir):
    """Takes in an input training directory and returns 
    
        train_data, train_labels
        test_data, test_labels"""
    training_files = map(lambda x: import_image(''.join([input_dir, '/', 'data_batch_', str(x)])), range(2, 6))
    
    train_data, train_labels = reduce(lambda x, y: (np.append(x[0], y[0], axis=0), np.append(x[1], y[1], axis=0)), training_files)
    test_data, test_labels = import_image('data/cifar-10-batches-py/test_batch')
    
    return train_data, train_labels, test_data, test_labels

In [24]:
def unpickle_and_encode(file):
    with open(file, 'rb') as fo:
        dic = pickle.load(fo, encoding='bytes')
    return {str(k, 'utf-8'): v for k, v in dic.items()}

In [25]:
def import_image(path):
    raw_image_data = unpickle_and_encode(path)
    
    images, labels = raw_image_data['data'], raw_image_data['labels']
    images = images.reshape((images.shape[0], 3, 32, 32)).transpose(0, 2, 3, 1) 
    
    return images, labels

In [26]:
def plot_image(img_object, label = None):

SyntaxError: unexpected EOF while parsing (<ipython-input-26-61577829969c>, line 1)

In [ ]:
train_data, train_labels, test_data, test_labels = load_files('data/cifar-10-batches-py')
label_encodings = unpickle_and_encode('data/cifar-10-batches-py/batches.meta')['label_names']

In [50]:
label_encodings

[b'airplane',
 b'automobile',
 b'bird',
 b'cat',
 b'deer',
 b'dog',
 b'frog',
 b'horse',
 b'ship',
 b'truck']

{b'num_cases_per_batch': 10000,
 b'label_names': [b'airplane',
  b'automobile',
  b'bird',
  b'cat',
  b'deer',
  b'dog',
  b'frog',
  b'horse',
  b'ship',
  b'truck'],
 b'num_vis': 3072}